# Gaussian Naive Bayes - Lab

## Introduction

Now that you've seen how to employ multinomial Bayes for classification, its time to practice implementing the process yourself. You'll also get a chance to investigate the impacts of using true probabilities under the probability density function as opposed to the point estimate on the curve itself.

## Objectives

You will be able to:

* Independently code and implement the Gaussian Naive Bayes algorithm

## Load the dataset

To get started, load the dataset stored in the file `'heart.csv'`. The dataset contains various measurements regarding patients and a `'target'` feature indicating whether or not they have heart disease. You'll be building a GNB classifier to help determine whether future patients do or do not have heart disease. As reference, this dataset was taken from Kaggle. You can see the original data post here: https://www.kaggle.com/ronitf/heart-disease-uci.

In [1]:
# Your code here 
# Load the dataset
import pandas as pd

df = pd.read_csv('data/heart.csv')
df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


## Define the problem

As discussed, the dataset contains various patient measurements along with a `'target'` variable indicating whether or not the individual has heart disease. Define `X` and `y` below: 

In [3]:
# Your code here
y = df.target
X = df.drop('target',axis=1)

## Perform a Train-test split

While not demonstrated in the previous lesson, you've seen from your work with regression that an appropriate methodology to determine how well your algorithm will generalize to new data is to perform a train-test split. 

> Note: Set `random_state` to 22 and `test_size` to 0.25 to have your results match those of the solution branch provided.

In [4]:
# Your code here
# Perform a train-test split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=22,test_size=0.25)

## Calculate the mean & standard deviation of each feature for each class in the training set

Now, calculate the mean and standard deviation for each feature within each of the target class groups. This will serve as your a priori distribution estimate to determine the posterior likelihood of an observation belonging to one class versus the other.

In [42]:
# Your code here 
# Calculate the mean and standard deviation for each feature within each class for the training set
train_df = pd.concat([X_train,y_train],axis=1)
train_agg_df = train_df.groupby('target').agg(['mean','std'])
train_agg_df

age                 sex                  cp            \
             mean       std      mean       std      mean       std   
target                                                                
0       57.281553  8.009085  0.796117  0.404853  0.466019  0.916253   
1       52.322581  9.995567  0.564516  0.497832  1.427419  0.972578   

          trestbps                   chol             ...     exang            \
              mean        std        mean        std  ...      mean       std   
target                                                ...                       
0       134.067961  18.919469  251.543689  52.341596  ...  0.543689  0.500523   
1       129.137097  16.589415  240.516129  46.683240  ...  0.161290  0.369291   

         oldpeak               slope                  ca                thal  \
            mean       std      mean       std      mean       std      mean   
target                                                                         
0       1.600971  1.310253  1.184466  0.555676  1.213592  1.025656  2.582524   
1       0.592742  0.749544  1.580645  0.612827  0.354839  0.818422  2.072581   

                  
             std  
target            
0       0.602678  
1       0.444906  

[2 rows x 26 columns]

## Define a function to calculate the point estimate for the conditional probability of a feature value for a given class

Recall that the point estimate is given by the probability density function of the normal distribution:  

 $$ \large P(x_i|y) = \frac{1}{\sqrt{2 \pi \sigma_i^2}}e^{\frac{-(x-\mu_i)^2}{2\sigma_i^2}}$$

> Note: Feel free to use the built-in function from SciPy to do this as demonstrated in the lesson. Alternatively, take the time to code the above formula from scratch.

###### My first solution

In [45]:
import numpy as np

In [102]:
# Your code here
def bayes_point_estimator(data:pd.DataFrame,target_header:str,target_negative=False):
    headers = list(data.columns)
    headers.remove(target_header)

    agg_df = data.groupby(target_header).agg(['mean','std'])
    
    Pex_dict = {}

    if target_negative:
        x = 0
    else:
        x = 1

    for head in headers:
        mu = agg_df[head]['mean'][x]
        sig = agg_df[head]['std'][x]

        base = 1/(np.sqrt(2*np.pi*np.power(sig,2))**np.e)
        power = (-(x-mu)**2)/(2*(sig**2))

        Pex_dict[head] = base**power

    return Pex_dict


In [109]:

bpe_true = bayes_point_estimator(train_df,'target')
bpe_true
    

{'age': 1.332837702226767e+50,
 'sex': 1.2589883278889178,
 'cp': 1.263542893023095,
 'trestbps': 1.878057620951295e+131,
 'chol': 9.935605600021201e+73,
 'fbs': 0.0898024617845519,
 'restecg': 1.1854853540529078,
 'thalach': 3.126215438729577e+144,
 'exang': 0.5819075736110609,
 'oldpeak': 1.2879404553517861,
 'slope': 1.6883731245641802,
 'ca': 1.834717893883577,
 'thal': 2.3664390638822184}

###### the solutiong code

In [161]:
from scipy import stats
def p_x_given_class(obs_row, feature, class_):
    mu = train_agg_df[feature]['mean'][class_]
    std = train_agg_df[feature]['std'][class_]
    
    # Observation
    obs = obs_row[feature] 
    
    p_x_given_y = stats.norm.pdf(obs, loc=mu, scale=std)
    return p_x_given_y
p_x_given_class(X_train.iloc[0], X.columns[0], 0)

0.035036938123834606

In [118]:
train_agg_df

age                 sex                  cp            \
             mean       std      mean       std      mean       std   
target                                                                
0       57.281553  8.009085  0.796117  0.404853  0.466019  0.916253   
1       52.322581  9.995567  0.564516  0.497832  1.427419  0.972578   

          trestbps                   chol             ...     exang            \
              mean        std        mean        std  ...      mean       std   
target                                                ...                       
0       134.067961  18.919469  251.543689  52.341596  ...  0.543689  0.500523   
1       129.137097  16.589415  240.516129  46.683240  ...  0.161290  0.369291   

         oldpeak               slope                  ca                thal  \
            mean       std      mean       std      mean       std      mean   
target                                                                         
0       1.600971  1.310253  1.184466  0.555676  1.213592  1.025656  2.582524   
1       0.592742  0.749544  1.580645  0.612827  0.354839  0.818422  2.072581   

                  
             std  
target            
0       0.602678  
1       0.444906  

[2 rows x 26 columns]

###### my adaptation of the solution code

In [137]:
def p_x_given_class(data:pd.DataFrame, feature:str, obs_idx:int, class_=0):
    headers = list(data.columns)
    headers.remove(feature)
    agg_df = data.groupby(feature).agg(['mean','std'])

    mu = train_agg_df[feature]['mean'][class_]
    sig = train_agg_df[feature]['std'][class_]

    Pxy_list = []

    for i in range(len(data)):
        obs = data.iloc[i][feature]
        p_x_given_y = stats.norm.pdf(obs, loc=mu, scale=sig)
        Pxy_list.append(p_x_given_y)

    return Pxy_list[obs_idx]

In [140]:
point_est = p_x_given_class(train_df,'age',0)
point_est

0.035036938123834606

## Define a prediction function 

Define a prediction function that will return a predicted class value for a particular observation. To do this, calculate the point estimates for each of the features using your function above. Then, take the product of these point estimates for a given class and multiply it by the probability of that particular class. Take the class associated with the largest probability output from these calculations as your prediction.

In [162]:
import numpy as np
def predict_class(obs_row):
    c_probs = []
    for c in range(2):
        # Initialize probability to relative probability of class
        p = len(y_train[y_train == c])/len(y_train) 
        for feature in X.columns:
            p *= p_x_given_class(obs_row, feature, c)
        c_probs.append(p)
    return np.argmax(c_probs)

In [163]:
predict_class(X_train.iloc[0])

0

## Apply your prediction function to the training and test sets

In [164]:
# Your code here
y_hat_train = [predict_class(X_train.iloc[idx]) for idx in range(len(X_train))]
y_hat_test = [predict_class(X_test.iloc[idx]) for idx in range(len(X_test))]

## Calculate the training and test accuracy

In [165]:
# Your code here
residuals_train = y_hat_train == y_train
acc_train = residuals_train.sum()/len(residuals_train)

residuals_test = y_hat_test == y_test
acc_test = residuals_test.sum()/len(residuals_test)
print('Training Accuracy: {}\tTesting Accuracy: {}'.format(acc_train, acc_test))

Training Accuracy: 0.8502202643171806	Testing Accuracy: 0.8289473684210527


## Level up (Optional)

### Adapting point estimates for the conditional probability into true probability estimates

As discussed, the point estimate from the probability density function is not a true probability measurement. Recall that the area under a probability density function is 1, representing the total probability of all possible outcomes. Accordingly, to determine the probability of a feature measurement occurring, you would need to find the area under some portion of the PDF. Determining appropriate bounds for this area however, is a bit tricky and arbitrary. For example, when generating a class prediction, you would want to know the probability of a patient having a resting blood pressure of 145 given that they had heart disease versus the probability of having a resting blood pressure of 145 given that the did not have heart disease. Previously, you've simply used the point where x=145 on the PDF curve to do this. However, the probability of any single point is actually 0. To calculate the actual probability, you would have to create a range around the observed value such as "what is the probability of having a resting blood pressure between 144 and 146 inclusive?" Alternatively, you could narrow the range and rewrite the problem as "what is the probability of having a resting blood pressure between 144.5 and 145.5?" Since defining these bounds is arbitrary, a potentially interesting research question is how various band methods might impact output predictions and the overall accuracy of the algorithm.


## Rewriting the conditional probability formula

Rewrite your conditional probability formula above to take a feature observation, a given class, and a range width and calculate the actual probability beneath the PDF curve of an observation falling within the range of the given width centered at the given observation value. For example, taking the previous example of resting blood pressure, you might calculate the probability of having a resting blood pressure within 1bp of 145 given that a patient has heart disease. In this case, the range width would be 2bp (144bp to 146bp) and the corresponding area under the PDF curve for the normal distribution would look like this:  

<img src="images/pdf_integral.png">

With that, write such a function below: 

In [ ]:
def p_band_x_given_class(obs_row, feature, c, range_width_std):
    """obs_row is the observation in question 
    feature is the feature of the observation row for which you are calculating a conditional probability 
    c is the class flag for the conditional probability 
    range_width_std is the range in standard deviations of the feature variable to calculate the integral under the PDF curve for"""
    # Your code here 
    
    return p_x_given_y

## Update the prediction function

Now, update the `predict_class()` function to use this new conditional probability function. Be sure that you can pass in the range width variable to this wrapper function.

In [ ]:
# Your code here
# Update the prediction function


## Experiment with the impact of various range-widths

Finally, write a `for` loop to measure the impact of varying range-widths on the classifier's test and train accuracy. Iterate over various range-widths from 0.1 standard deviations to 2 standard deviations. For each of these, store the associated test and train accuracies. Finally, plot these on a graph. The x-axis should be the associated range-width (expressed in standard deviations; each feature will have a unique width applicable to the specific scale). The y-axis will be the associated accuracy. Be sure to include a legend for train accuracy versus test accuracy.

_Note:_ ⏰ _Expect your code to take over two minutes to run._

In [ ]:
# Your code here


> Comment: Not a wild difference from our point estimates obtained by using points from the PDF itself, but there is some impact. **Interestingly, these graphs will differ substantially in shape depending on the initial train-test split used.** The recommendation would be to use the point estimates from the PDF itself, or a modest band-width size.

## Appendix: Plotting PDFs and probability integrals

Below, feel free to take a look at the code used to generate the PDF graph image above.

In [ ]:
temp = df[df['target'] == 1]['trestbps']
aggs = temp.agg(['mean', 'std'])
aggs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import seaborn as sns
import scipy.stats as stats
sns.set_style('white')

In [ ]:
x = np.linspace(temp.min(), temp.max(), num=10**3)
pdf = stats.norm.pdf(x, loc=aggs['mean'], scale=aggs['std'])
xi = 145
width = 2
xi_lower = xi - width/2
xi_upper = xi + width/2

fig, ax = plt.subplots()

plt.plot(x, pdf)

# Make the shaded region
ix = np.linspace(xi_lower, xi_upper)
iy = stats.norm.pdf(ix, loc=aggs['mean'], scale=aggs['std'])
verts = [(xi_lower, 0), *zip(ix, iy), (xi_upper, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly);

plt.plot((145, 145), (0, stats.norm.pdf(145, loc=aggs['mean'], scale=aggs['std'])), linestyle='dotted')
p_area = stats.norm.cdf(xi_upper, loc=aggs['mean'], scale=aggs['std']) - stats.norm.cdf(xi_lower, loc=aggs['mean'], scale=aggs['std'])
print('Probability of Blood Pressure Falling withing Range for the Given Class: {}'.format(p_area))
plt.title('Conditional Probability of Resting Blood Pressure ~145 for Those With Heart Disease')
plt.ylabel('Probability Density')
plt.xlabel('Resting Blood Pressure')

> Comment: See https://matplotlib.org/gallery/showcase/integral.html for further details on plotting shaded integral areas under curves.

## Summary

Well done! In this lab, you implemented the Gaussian Naive Bayes classifier from scratch, and used it to generate classification predictions and validated the accuracy of the model.